# Lecture 10: Diffusion and dispersion errors



In [1]:
# Initialisation

%matplotlib inline

import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from scipy.sparse import diags
from ipywidgets import *
from IPython.display import display, Markdown

## Upwind Euler forward scheme of linear advection equation

The numerical amplication factor for this discretisation is

$$
G = (1-\sigma+\sigma\cos(\phi)) - I \sigma \sin(\phi)
$$

for $\sigma=\frac{a\Delta t}{\Delta x}$ and $\phi = \phi_m = k_m \Delta x$

In [2]:
phi_m = Symbol(r'\phi_m', real=True)
sigma = Symbol(r'\sigma', real=True)
y = Symbol('y')

G = (1-sigma+sigma*cos(phi_m))-I*sigma*sin(phi_m)
r = np.linspace(0,2*np.pi,num=50)
u1 = [sin(x) for x in r]
u2 = [cos(x) for x in r]

def update1(s=0.5):
    Gt = G.subs(sigma, s)
    Gr = [re(Gt).subs(phi_m, x) for x in r]
    Gi = [im(Gt).subs(phi_m, x) for x in r]
    
    fig = plt.figure()
    ax = plt.axes(xlim=(-1.9, 1.1),ylim=(-1.1,1.1))

    plt.plot(u1,u2, color='tab:blue')
    plt.plot(Gr,Gi, color='tab:orange')
    plt.grid()
    plt.legend(['Unit Circle', 'Locus'])
    
    # Plot arrows
    plt.annotate(s='', xy=(-sqrt(1/2),sqrt(1/2)), xytext=(0,0), arrowprops=dict(arrowstyle='->', shrinkA=0, shrinkB=0, color='tab:blue'))
    plt.annotate(s='', xy=(Gr[42],Gi[42]), xytext=(Gr[42],0), arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0, color='tab:red'))
    plt.annotate(s='', xy=(0,0), xytext=(Gr[42],0), arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0, color='tab:red'))
    plt.annotate(s='', xy=(Gr[42],Gi[42]), xytext=(0,0), arrowprops=dict(arrowstyle='->', shrinkA=0, shrinkB=0, color='tab:green'))
    

interact1 = interact(update1, s=FloatSlider(min=0.1, max=1.1, value=0.5, step=0.05, continuous_update=False, description='$\sigma$'))

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma$', max=1.1, min=0.…

As we have seen the *exact amplification factor* is $\tilde{G} = e^{-Ik_ma\Delta t}$, we can then find that $|\tilde{G}|=1$ and $\tilde{\Phi}=k_ma\Delta t$ .

We then find for the first order upwind scheme:

$$
\varepsilon_D = \left[ 1 - 4\sigma(1-\sigma)\sin^2(\phi/2) \right]^{1/2}
$$

$$
\varepsilon_{\Phi} = \frac{\tan^{-1}[\sigma\sin(\phi)/(1-\sigma+\sigma\cos(\phi))]}{\phi\sigma}
$$


In [3]:
def update2(s=0.5):
    r = np.linspace(0,np.pi,num=26)
    ed = (1-4*sigma*(1-sigma)*(sin(phi_m/2))**2)**(1/2)
    ep = (atan2((sigma*sin(phi_m)),(1-sigma+sigma*cos(phi_m))))/(phi_m*sigma)
    eds = ed.subs(sigma,s)
    eps = ep.subs(sigma,s)
    
    fig2 = plt.figure()
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'$\varepsilon$')

    plt.plot(r,[eds.subs(phi_m, x) for x in r], color='tab:blue')
    plt.plot(r,[eps.subs(phi_m, x) for x in r], color='tab:orange')
    plt.grid()
    plt.legend([r'$\varepsilon_D$',r'$\varepsilon_\phi$'])
    

interact2 = interact(update2, s=FloatSlider(min=0.1, max=1.0, value=0.5, step=0.05, continuous_update=False, description='$\sigma$'))

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma$', max=1.0, min=0.…

## Error analysis for the diffusion equation

The numerical amplication factor for this discretisation, as seen earlier, is

$$
G = 1 - 4 \sigma \sin^2(\phi/2)
$$

Hence, $|G|= 1 - 4 \sigma \sin^2(\phi/2)$ and $\Phi=0$, i.e. $\varepsilon_\Phi = 0$ and there is no dispersion error in this scheme.

In [4]:
def update3(s=0.5):
    r = np.linspace(0,np.pi,num=26)
    ed = (1-4*sigma*(sin(phi_m/2))**2)/(exp(-sigma*phi_m**2))
    eds = ed.subs(sigma,s)
    
    fig3 = plt.figure()
    ax3 = plt.gca()
    ax3.set_ylim([-4,2])
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'$\varepsilon$')

    plt.plot(r,[eds.subs(phi_m, x) for x in r], color='tab:blue')
    plt.grid()
    plt.legend([r'$\varepsilon_D$'], loc='lower left')
    

interact3 = interact(update3, s=FloatSlider(min=0.0, max=0.5, value=0.2, step=0.05, continuous_update=False, description='$\sigma$'))

interactive(children=(FloatSlider(value=0.2, continuous_update=False, description='$\\sigma$', max=0.5, step=0…

## Leapfrog Scheme for linear advection equation

The **leapfrog scheme** is an explicit scheme which is second order in both time and space, defined by

$$
\frac{u^{n+1}_i - u^{n-1}_i}{2 \Delta t} + \frac{a}{2 \Delta x} \left( u^n_{i+1} - u^n_{i-1} \right) = 0
$$

We can then obtain a quadratic equation for G as such

$$
G - \frac{1}{G} = -\sigma \left( e^{I \phi} - e^{-I \phi} \right)
$$

with the solutions

$$
G = -I \sigma \sin \phi \pm \sqrt{1 - \sigma^2 \sin^2 \phi }
$$

If $\sigma > 1$, the scheme is unstable as the term under the square root can become negative, making G purely imaginary and larger than one in magnitude.

For $|\sigma| \leq 1$, the term under the square root is always real and thus

$$
|G|^2 = \sigma^2 \sin^2 \phi + \left( 1 - \sigma^2 \sin^2 \phi \right)
$$

$$
|G| = 1 \quad \textrm{for } |\sigma| \leq 1
$$

Thus, the leapfrog scheme is **neutrally stable**, i.e. the scheme has no diffusion errors.

The dispersion error is given by

$$
\varepsilon_{\Phi} = \pm \frac{\tan^{-1}\left[ \sigma\sin(\phi) \middle/ \sqrt{1 -  \sigma^2 \sin^2 \phi} \right]}{\phi\sigma} = \pm \frac{\sin^{-1} (\sigma \sin \phi)}{\sigma \phi}
$$

We compare the dispersion error for the leapfrog scheme to the upwind Euler scheme.

In [5]:
def update4(s=0.5):
    r = np.linspace(0,np.pi,num=26)
    # Leapfrog scheme
    epl = (asin(sigma*sin(phi_m)))/(sigma*phi_m)
    epls = epl.subs(sigma,s)
    # Forward Euler
    epf = (atan2((sigma*sin(phi_m)),(1-sigma+sigma*cos(phi_m))))/(phi_m*sigma)
    epfs = epf.subs(sigma,s)
    
    fig3 = plt.figure()
    ax3 = plt.gca()
    ax3.set_ylim([0,1.4])
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'$\varepsilon$')

    plt.plot(r,[epls.subs(phi_m, x) for x in r], color='tab:blue')
    plt.plot(r,[epfs.subs(phi_m, x) for x in r], color='tab:orange')
    plt.grid()
    plt.legend(['Leapfrog', 'Forward Euler'], loc='lower left')
    

interact4 = interact(update4, s=FloatSlider(min=0.05, max=1., value=0.2, step=0.05, continuous_update=False, description='$\sigma$'))

interactive(children=(FloatSlider(value=0.2, continuous_update=False, description='$\\sigma$', max=1.0, min=0.…